In [18]:
#!pip3 install pulp
#!pip3 install pandas

In [19]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable,LpBinary
import pulp

In [20]:
items = pd.read_csv("../data/rei_items.csv")
items.head()


,item_id,item_name,link,brand,review_cnt,avg_rating,low_price,high_price,weight_lbs,volume_L,temp,type,sleeping_bag,backpack,tent
0,162948,NEMO Forte 20 Sleeping Bag - Men's,https://www.rei.com/product/162948/nemo-forte-...,NEMO,156,4.3,149.89,164.89,2.8750,9.0,20.0,Sleeping Bag,1,0,0
1,157769,REI Co-op Trailbreak 20 Sleeping Bag - Men's,https://www.rei.com/product/157769/rei-co-op-t...,REI,112,4.5,64.93,109.00,3.4375,12.0,20.0,Sleeping Bag,1,0,0
2,218408,Big Agnes Torchlight Camp 20 Sleeping Bag - Men's,https://www.rei.com/rei-garage/product/218408/...,Big Agnes,5,4.6,134.73,179.95,3.0625,9.3,20.0,Sleeping Bag,1,0,0
3,168251,REI Co-op Trailbreak 60 Pack - Men's,https://www.rei.com/product/168251/rei-co-op-t...,REI,53,4.5,149,149.00,3.8125,60.0,NaN,Backpacking Packs,0,1,0
4,201301,Osprey Atmos AG 50 Pack - Men's,https://www.rei.com/product/201301/osprey-atmo...,Osprey,7,4.1,300,300.00,4.3125,50.0,NaN,Backpacking Packs,0,1,0


### What is the problem we want to solve?

Objective Function: Minimize Cost + Weight
- z = ac + aw + a + bc + bw + b + ...

Constaints:
Minimum of one item per category 
 

In [21]:
model = LpProblem(name="small-problem", sense=LpMinimize)

In [22]:
# Initialize the decision variables (all are continuous, except maybe lPvarible)
a = LpVariable(name="a", lowBound=0, cat="Binary")
a_c = LpVariable(name="a_c", lowBound=0)
a_w = LpVariable(name="a_w", lowBound=0)

b = LpVariable(name="b", lowBound=0, cat="Binary")
b_c = LpVariable(name="b_c", lowBound=0)
b_w = LpVariable(name="b_w", lowBound=0)

c = LpVariable(name="c", lowBound=0, cat="Binary")
c_c = LpVariable(name="c_c", lowBound=0)
c_w = LpVariable(name="c_w", lowBound=0)


In [23]:
## Add input variables 
max_cost = 1000
max_weight = 10

In [24]:
# Add the constraints to the model
model += (a == 1, "only one a ")
model += (b == 1, "only one b ")
model += (c == 1, "only one c ")

model += (a_c + b_c + c_c <= max_cost, "cost_constraint")
model += (a_w + b_w + c_w <= max_weight, "weight_constraint")


In [25]:
# Add the objective function to the model
model += lpSum([a, a_c, a_w, b, b_c, b_w, c, c_c, c_w])



In [26]:
model

small-problem:
MINIMIZE
1*a + 1*a_c + 1*a_w + 1*b + 1*b_c + 1*b_w + 1*c + 1*c_c + 1*c_w + 0
SUBJECT TO
only_one_a_: a = 1

only_one_b_: b = 1

only_one_c_: c = 1

cost_constraint: a_c + b_c + c_c <= 1000

weight_constraint: a_w + b_w + c_w <= 10

VARIABLES
0 <= a <= 1 Integer
a_c Continuous
a_w Continuous
0 <= b <= 1 Integer
b_c Continuous
b_w Continuous
0 <= c <= 1 Integer
c_c Continuous
c_w Continuous

In [27]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
model.solver

status: 1, Optimal
objective: 3.0


## Now real attempt 'to balance your bag'

In [28]:
## Add input variables 
max_cost = 1000
max_weight = 8

In [29]:
# create pivot id to separate line items into group types 
items['type_id'] = items.groupby('type').ngroup().astype('str') + '_' + items.groupby('type').cumcount().astype('str')
# add standing flag for id of single entry per group
items['single_flg'] = 1

In [30]:
pd.options.display.max_columns = None

item_matrix = items[['type','type_id','item_id','single_flg','high_price','weight_lbs']].pivot(index='type_id',columns='type',values=['single_flg','item_id','high_price','weight_lbs'])
item_matrix.columns = list(map("_".join, item_matrix.columns))
item_matrix = item_matrix.fillna(0)
item_matrix


#type_id can be item_id



,single_flg_Backpacking Packs,single_flg_Sleeping Bag,single_flg_Tent,item_id_Backpacking Packs,item_id_Sleeping Bag,item_id_Tent,high_price_Backpacking Packs,high_price_Sleeping Bag,high_price_Tent,weight_lbs_Backpacking Packs,weight_lbs_Sleeping Bag,weight_lbs_Tent
type_id,,,,,,,,,,,,
0_0,1.0,0.0,0.0,168251.0,0.0,0.0,149.0,0.00,0.00,3.8125,0.0000,0.000
0_1,1.0,0.0,0.0,201301.0,0.0,0.0,300.0,0.00,0.00,4.3125,0.0000,0.000
0_2,1.0,0.0,0.0,177493.0,0.0,0.0,315.0,0.00,0.00,5.1250,0.0000,0.000
1_0,0.0,1.0,0.0,0.0,162948.0,0.0,0.0,164.89,0.00,0.0000,2.8750,0.000
1_1,0.0,1.0,0.0,0.0,157769.0,0.0,0.0,109.00,0.00,0.0000,3.4375,0.000
1_2,0.0,1.0,0.0,0.0,218408.0,0.0,0.0,179.95,0.00,0.0000,3.0625,0.000
2_0,0.0,0.0,1.0,0.0,0.0,147862.0,0.0,0.00,329.00,0.0000,0.0000,2.375
2_1,0.0,0.0,1.0,0.0,0.0,168431.0,0.0,0.00,139.00,0.0000,0.0000,4.625
2_2,0.0,0.0,1.0,0.0,0.0,188351.0,0.0,0.00,399.95,0.0000,0.0000,2.125


In [31]:

    # Setup problem 
    Leveler = LpProblem("Leveler", LpMinimize)

    # GENERATE ID AND OBJECTIVE FUNCTION VARIABLES 
    type_id = list(item_matrix.index)

    type_a_flg = dict(zip(type_id, item_matrix['single_flg_Backpacking Packs']))
    type_b_flg = dict(zip(type_id, item_matrix['single_flg_Sleeping Bag']))
    type_c_flg = dict(zip(type_id, item_matrix['single_flg_Tent']))

    type_a_dollars = dict(zip(type_id, item_matrix['high_price_Backpacking Packs']))
    type_b_dollars = dict(zip(type_id, item_matrix['high_price_Sleeping Bag']))
    type_c_dollars = dict(zip(type_id, item_matrix['high_price_Tent']))

    type_a_weight = dict(zip(type_id, item_matrix['weight_lbs_Backpacking Packs']))
    type_b_weight = dict(zip(type_id, item_matrix['weight_lbs_Sleeping Bag']))
    type_c_weight = dict(zip(type_id, item_matrix['weight_lbs_Tent']))

    # # Create the variables to indicate if an item has been selected
    bag_vars_b = LpVariable.dicts("InOrOut", type_id, 0, 1, LpBinary)
    # # Initiate objective function (Min dollars and weight for selected variables in opps)
    Leveler += lpSum([type_a_dollars[i]*bag_vars_b[i] + type_b_dollars[i]*bag_vars_b[i] + type_c_dollars[i]*bag_vars_b[i] 
                    + type_a_weight[i]*bag_vars_b[i] + type_b_weight[i]*bag_vars_b[i] + type_c_weight[i]*bag_vars_b[i] for i in type_id])

    Leveler += lpSum([type_a_flg[f] * bag_vars_b[f] for f in type_id]) == 1, 'single_item_a'
    Leveler += lpSum([type_b_flg[f] * bag_vars_b[f] for f in type_id]) == 1, 'single_item_b'
    Leveler += lpSum([type_c_flg[f] * bag_vars_b[f] for f in type_id]) == 1, 'single_item_c'

    Leveler += lpSum([type_a_dollars[i]*bag_vars_b[i] + type_b_dollars[i]*bag_vars_b[i] + type_c_dollars[i]*bag_vars_b[i] for i in type_id]) <= max_cost, 'price_limit'
    Leveler += lpSum([type_a_weight[i]*bag_vars_b[i] + type_b_weight[i]*bag_vars_b[i] + type_c_weight[i]*bag_vars_b[i] for i in type_id]) <= max_weight, 'weight_limit'

Leveler
    

Leveler:
MINIMIZE
152.8125*InOrOut_0_0 + 304.3125*InOrOut_0_1 + 320.125*InOrOut_0_2 + 167.765*InOrOut_1_0 + 112.4375*InOrOut_1_1 + 183.0125*InOrOut_1_2 + 331.375*InOrOut_2_0 + 143.625*InOrOut_2_1 + 402.075*InOrOut_2_2 + 0.0
SUBJECT TO
single_item_a: InOrOut_0_0 + InOrOut_0_1 + InOrOut_0_2 = 1

single_item_b: InOrOut_1_0 + InOrOut_1_1 + InOrOut_1_2 = 1

single_item_c: InOrOut_2_0 + InOrOut_2_1 + InOrOut_2_2 = 1

price_limit: 149 InOrOut_0_0 + 300 InOrOut_0_1 + 315 InOrOut_0_2
 + 164.89 InOrOut_1_0 + 109 InOrOut_1_1 + 179.95 InOrOut_1_2 + 329 InOrOut_2_0
 + 139 InOrOut_2_1 + 399.95 InOrOut_2_2 <= 1000

weight_limit: 3.8125 InOrOut_0_0 + 4.3125 InOrOut_0_1 + 5.125 InOrOut_0_2
 + 2.875 InOrOut_1_0 + 3.4375 InOrOut_1_1 + 3.0625 InOrOut_1_2
 + 2.375 InOrOut_2_0 + 4.625 InOrOut_2_1 + 2.125 InOrOut_2_2 <= 8

VARIABLES
0 <= InOrOut_0_0 <= 1 Integer
0 <= InOrOut_0_1 <= 1 Integer
0 <= InOrOut_0_2 <= 1 Integer
0 <= InOrOut_1_0 <= 1 Integer
0 <= InOrOut_1_1 <= 1 Integer
0 <= InOrOut_1_2 <= 1 Intege

In [32]:
    Leveler.solve(pulp.PULP_CBC_CMD(timeLimit=300, msg=1, gapRel=0))


-1

In [33]:
    balanced_bag = []
    for v in Leveler.variables():
        if v.varValue > 0:
            balanced_bag.append(v.name)
    balanced_bag = [s.replace('InOrOut_', "") for s in balanced_bag]
    # export to data 
    
    all_balanced_bag = items.loc[items['type_id'].astype(str).isin(balanced_bag)]
    all_balanced_bag
    

,item_id,item_name,link,brand,review_cnt,avg_rating,low_price,high_price,weight_lbs,volume_L,temp,type,sleeping_bag,backpack,tent,type_id,single_flg
0,162948,NEMO Forte 20 Sleeping Bag - Men's,https://www.rei.com/product/162948/nemo-forte-...,NEMO,156,4.3,149.89,164.89,2.8750,9.0,20.0,Sleeping Bag,1,0,0,1_0,1
3,168251,REI Co-op Trailbreak 60 Pack - Men's,https://www.rei.com/product/168251/rei-co-op-t...,REI,53,4.5,149,149.00,3.8125,60.0,NaN,Backpacking Packs,0,1,0,0_0,1
8,188351,Big Agnes Tiger Wall UL 1 Solution-Dyed Tent,https://www.rei.com/product/188351/big-agnes-t...,Big Agnes,7,4.3,399.95,399.95,2.1250,NaN,NaN,Tent,0,0,1,2_2,1
